This notebook includes the use of five different classification models (VGG16, ResNet50, Inception V3, Densenet121, 4 Layer CNN) to distinguish between Alzheimer's and healthy controls using scalogram images from all lobes. The models were trained and evaluated using all the epochs (segments) of the images.

*Fatemeh Makhloughi*

In [2]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, KFold
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import Sequential, Model, clone_model, load_model
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.layers import Conv2D, Input, ConvLSTM2D, MaxPooling3D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D, LSTM, GRU, TimeDistributed
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, EfficientNetB2
import shutil

ImportError: cannot import name 'OrderedDict' from 'typing' (C:\Users\BioElectric\AppData\Local\Programs\Python\Python37\lib\typing.py)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### In local computer

In [2]:
import os
import re
import numpy as np

# Define the local directory path
data_dir = os.path.join('F:', 'Abdipour', 'Alzheimer', 'Spectrogram Images', 'Temporal')

# Function to extract the numeric part of the folder name
def extract_numeric_part(folder_name):
    # Use a regular expression to find the numeric part at the end of the folder name
    match = re.search(r'\d+$', folder_name)
    if match:
        return int(match.group())
    else:
        raise ValueError(f"No numeric part found in folder name: {folder_name}")

# List all directories in the specified directory and sort them based on the numeric part
folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

subject_folders, labels = [], []

# Iterate through the sorted folders
for folder_name in folders:
    # Extract the numeric part from the folder name
    subject_num = extract_numeric_part(folder_name)
    subject_folder = os.path.join(data_dir, folder_name)
    subject_folders.append(subject_folder)

    # Assign labels based on the numeric part
    if subject_num <= 36:
        labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        labels.append(1)  # HC group

# Convert lists to numpy arrays
subject_folders = np.array(subject_folders)
labels = np.array(labels)

# Print the results (optional)
print("Subject Folders:", subject_folders)
print("Labels:", labels)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'F:Abdipour\\Alzheimer\\Spectrogram Images\\Temporal'

# 1Hz- Temporal Lobe- Ignore Concatenate

In [ ]:
import os
import re
import numpy as np

# Define the directory path
data_dir = os.path.join('/content/drive/My Drive/', 'Alzheimer', 'Spectrogram Images', 'Temporal')

# Function to extract the numeric part of the folder name
def extract_numeric_part(folder_name):
    # Use a regular expression to find the numeric part at the end of the folder name
    match = re.search(r'\d+$', folder_name)
    if match:
        return int(match.group())
    else:
        raise ValueError(f"No numeric part found in folder name: {folder_name}")

# List all directories in the specified directory and sort them based on the numeric part
folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

subject_folders, labels = [], []

# Iterate through the sorted folders
for folder_name in folders:
    # Extract the numeric part from the folder name
    subject_num = extract_numeric_part(folder_name)
    subject_folder = os.path.join(data_dir, folder_name)
    subject_folders.append(subject_folder)

    # Assign labels based on the numeric part
    if subject_num <= 36:
        labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        labels.append(1)  # HC group

# Convert lists to numpy arrays
subject_folders = np.array(subject_folders)
labels = np.array(labels)

# Print the results (optional)
print("Subject Folders:", subject_folders)
print("Labels:", labels)


Subject Folders: ['/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject1'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject2'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject3'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject4'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject5'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject6'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject7'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject8'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject9'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject10'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject11'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject12'
 '/content/drive/My Drive/Alzheimer/Spectrogram Images/Temporal/subject13'
 '/content/drive/

In [ ]:
def load_images_and_labels(subjects, labels):
    images, labels_t = [], []
    for subject_folder, label in zip(subjects, labels):
        for filename in os.listdir(subject_folder):
                img_path = os.path.join(subject_folder, filename.decode())
                try:
                    img = load_img(img_path, target_size=(224, 224))
                    img_array = img_to_array(img)
                    img_array = img_array / 255.0
                    images.append(img_array)
                    labels_t.append(label)
                except Exception as e:
                    print(f"Error loading image: {img_path}, {e}")

    return np.array(images), np.array(labels_t)

In [ ]:
def load_and_preprocess_data(subject_folders, labels):
    train_subjects, test_subjects, train_labels, test_labels = train_test_split(subject_folders, labels, test_size=0.2, stratify=labels, random_state = 42)

    train_subjects, val_subjects, train_labels, val_labels = train_test_split(train_subjects, train_labels, test_size=0.1, stratify=train_labels, random_state = 42)

    train_images, train_labels_t = load_images_and_labels(train_subjects, train_labels)
    val_images, val_labels_t = load_images_and_labels(val_subjects, val_labels)

    test_image_counts_per_subject = [len(os.listdir(folder)) for folder in test_subjects]

    test_images, test_labels_t = load_images_and_labels(test_subjects, test_labels)

    return train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject


In [ ]:
def create_new_model(base_model):
    model = clone_model(base_model)

    for layer in model.layers:
        layer.trainable = False

    x = model.output
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Dropout(0.2)(x)
    x = Dense(256)(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = Dropout(0.2)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=model.input, outputs=predictions)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])

    return model

In [ ]:
def train_and_evaluate_model(base_model, subject_folders, labels, num_epochs=50, num_iterations=5, model_name='model'):
    accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}:")

        model = create_new_model(base_model)

        train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(subject_folders, labels)

        train_data = list(zip(train_images, train_labels_t))
        np.random.shuffle(train_data)
        train_images, train_labels_t = zip(*train_data)

        val_data = list(zip(val_images, val_labels_t))
        np.random.shuffle(val_data)
        val_images, val_labels_t = zip(*val_data)

        # test_data = list(zip(test_images, test_labels_t))
        # np.random.shuffle(test_data)
        # test_images, test_labels_t = zip(*test_data)

        train_images, train_labels_t = np.array(train_images), np.array(train_labels_t)
        val_images, val_labels_t = np.array(val_images), np.array(val_labels_t)
        # test_images, test_labels_t = np.array(test_images), np.array(test_labels_t)



        # checkpoint_filepath = "best_weights_iteration_{iteration + 1}.h5"
        # checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
        #                                        monitor='val_accuracy',
        #                                        mode='max',
        #                                        save_best_only=True,
        #                                        save_weights_only=True,
        #                                        verbose=1)

        # history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=8, validation_data=(val_images, val_labels_t), callbacks=[checkpoint], verbose=1)

        checkpoint_dir = '/content'
        os.makedirs(checkpoint_dir, exist_ok=True)

        local_checkpoint_filepath = os.path.join(checkpoint_dir, f"{model_name}_best_weights_iteration_{iteration + 1}.h5")
        drive_checkpoint_filepath = f'/content/drive/My Drive/{model_name}_best_weights_iteration_{iteration + 1}.h5'

        checkpoint_callback = ModelCheckpoint(filepath=local_checkpoint_filepath,
                                               monitor='val_accuracy',
                                               mode='max',
                                               save_best_only=True,
                                               save_weights_only=True,
                                               verbose=1)

        early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)

        history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=32, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping_callback], verbose=1)

        if os.path.exists(local_checkpoint_filepath):
            print(f"Best weights saved locally at: {local_checkpoint_filepath}")

            shutil.copy(local_checkpoint_filepath, drive_checkpoint_filepath)
            if os.path.exists(drive_checkpoint_filepath):
                print(f"Best weights also copied to Google Drive at: {drive_checkpoint_filepath}")
            else:
                print(f"Failed to copy best weights to Google Drive at: {drive_checkpoint_filepath}")
        else:
            print(f"Failed to save best weights locally at: {local_checkpoint_filepath}")

        model.load_weights(local_checkpoint_filepath)

        test_predictions = model.predict(test_images)
        test_predictions = (test_predictions > 0.5).astype(int)

        accuracy = accuracy_score(test_labels_t, test_predictions)
        precision = precision_score(test_labels_t, test_predictions)
        recall = recall_score(test_labels_t, test_predictions)
        f1 = f1_score(test_labels_t, test_predictions)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        print("ModelCheckpoint callback triggered and saved the best weights.")


        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_accuracy_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)



        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_loss_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)


        cm = confusion_matrix(test_labels_t, test_predictions)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['AD', 'HC'], yticklabels=['AD', 'HC'])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {model_name} - Iteration {iteration + 1}')
        file_name = f'{model_name}_confusion_matrix_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)




        # print("\nPredictions for Each Subject's Images:")
        # start_idx = 0
        # for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
        #     end_idx = start_idx + num_images
        #     subject_images = test_images[start_idx:end_idx]
        #     subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
        #     print(f"Subject {i + 1} ({subject_folder}):")
        #     for j, prediction in enumerate(subject_predictions):
        #         if prediction == 0:
        #             print(f"Image {j + 1}: AD")
        #         else:
        #             print(f"Image {j + 1}: HC")
        #     start_idx = end_idx


        print("\nPredictions for Each Subject's Images:")
        start_idx = 0
        for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_images[start_idx:end_idx]
            subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx




    print("\nAverage Metrics:")
    print(f"Average Accuracy: {np.mean(accuracy_list):.4f}")
    print(f"Average Precision: {np.mean(precision_list):.4f}")
    print(f"Average Recall: {np.mean(recall_list):.4f}")
    print(f"Average F1 Score: {np.mean(f1_list):.4f}")


## VGG16

In [ ]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
    # x = model.output
    # x = Flatten()(x)
    # x = Dense(128)(x)
    # x = Dropout(0.2)(x)
    # x = Dense(256)(x)
    # x = Dropout(0.2)(x)
model_name = 'vgg16_1Hz_Temporal'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

## ResNet50

In [ ]:
ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
model_name = 'ResNet50_Temporal'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

## Densenet121

In [ ]:
Dense_base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
model_name = 'Densenet121_Temporal'
train_and_evaluate_model(Dense_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

## Inception V3

In [ ]:
Inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
model_name = 'InceptionV3_Temporal'
train_and_evaluate_model(Inception_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

## 4 layer CNN

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_new_model(input_shape):
    base_model = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(base_model)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model, outputs=predictions)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])

    return model


In [ ]:
def train_and_evaluate_model(subject_folders, labels, num_epochs=50, num_iterations=5, model_name='model'):
    accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}:")
        input_shape = (224, 224, 3)

        model = create_new_model(input_shape)

        train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(subject_folders, labels)

        train_data = list(zip(train_images, train_labels_t))
        np.random.shuffle(train_data)
        train_images, train_labels_t = zip(*train_data)

        val_data = list(zip(val_images, val_labels_t))
        np.random.shuffle(val_data)
        val_images, val_labels_t = zip(*val_data)

        # test_data = list(zip(test_images, test_labels_t))
        # np.random.shuffle(test_data)
        # test_images, test_labels_t = zip(*test_data)

        train_images, train_labels_t = np.array(train_images), np.array(train_labels_t)
        val_images, val_labels_t = np.array(val_images), np.array(val_labels_t)
        # test_images, test_labels_t = np.array(test_images), np.array(test_labels_t)




        # checkpoint_filepath = "best_weights_iteration_{iteration + 1}.h5"
        # checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
        #                                        monitor='val_accuracy',
        #                                        mode='max',
        #                                        save_best_only=True,
        #                                        save_weights_only=True,
        #                                        verbose=1)

        # history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=8, validation_data=(val_images, val_labels_t), callbacks=[checkpoint], verbose=1)


        checkpoint_dir = '/content'
        os.makedirs(checkpoint_dir, exist_ok=True)


        local_checkpoint_filepath = os.path.join(checkpoint_dir, f"{model_name}_best_weights_iteration_{iteration + 1}.h5")
        drive_checkpoint_filepath = f'/content/drive/My Drive/{model_name}_best_weights_iteration_{iteration + 1}.h5'


        checkpoint_callback = ModelCheckpoint(filepath=local_checkpoint_filepath,
                                               monitor='val_accuracy',
                                               mode='max',
                                               save_best_only=True,
                                               save_weights_only=True,
                                               verbose=1)


        early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)

        history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=8, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping_callback], verbose=1)


        if os.path.exists(local_checkpoint_filepath):
            print(f"Best weights saved locally at: {local_checkpoint_filepath}")


            shutil.copy(local_checkpoint_filepath, drive_checkpoint_filepath)
            if os.path.exists(drive_checkpoint_filepath):
                print(f"Best weights also copied to Google Drive at: {drive_checkpoint_filepath}")
            else:
                print(f"Failed to copy best weights to Google Drive at: {drive_checkpoint_filepath}")
        else:
            print(f"Failed to save best weights locally at: {local_checkpoint_filepath}")


        model.load_weights(local_checkpoint_filepath)


        test_predictions = model.predict(test_images)
        test_predictions = (test_predictions > 0.5).astype(int)

        accuracy = accuracy_score(test_labels_t, test_predictions)
        precision = precision_score(test_labels_t, test_predictions)
        recall = recall_score(test_labels_t, test_predictions)
        f1 = f1_score(test_labels_t, test_predictions)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")


        print("ModelCheckpoint callback triggered and saved the best weights.")


        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_accuracy_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)



        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_loss_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)


        cm = confusion_matrix(test_labels_t, test_predictions)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['AD', 'HC'], yticklabels=['AD', 'HC'])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {model_name} - Iteration {iteration + 1}')
        file_name = f'{model_name}_confusion_matrix_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)




        # print("\nPredictions for Each Subject's Images:")
        # start_idx = 0
        # for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
        #     end_idx = start_idx + num_images
        #     subject_images = test_images[start_idx:end_idx]
        #     subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
        #     print(f"Subject {i + 1} ({subject_folder}):")
        #     for j, prediction in enumerate(subject_predictions):
        #         if prediction == 0:
        #             print(f"Image {j + 1}: AD")
        #         else:
        #             print(f"Image {j + 1}: HC")
        #     start_idx = end_idx


        print("\nPredictions for Each Subject's Images:")
        start_idx = 0
        for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_images[start_idx:end_idx]
            subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx




    print("\nAverage Metrics:")
    print(f"Average Accuracy: {np.mean(accuracy_list):.4f}")
    print(f"Average Precision: {np.mean(precision_list):.4f}")
    print(f"Average Recall: {np.mean(recall_list):.4f}")
    print(f"Average F1 Score: {np.mean(f1_list):.4f}")


In [ ]:
train_and_evaluate_model(subject_folders, labels, num_epochs=100, num_iterations=5, model_name='4LCNN_Temporal')